# Demo parameter sweeps of channel measurements
This demonstrates the construction of larger datasets built by looping across measurements of different channels.

In [1]:
import channel_analysis
import xarray as xr

capture = channel_analysis.FilteredCapture(
    sample_rate=15.36e6, analysis_bandwidth=10e6, duration=0.2
)

analysis_spec = {
    'channel_power_time_series': {
        'detector_period': 10e-3,
        'power_detectors': ('rms', 'peak'),
    },
    'cyclic_channel_power': {
        'cyclic_period': 10e-3,
        'detector_period': 1e-3 / 15 / 4,
        'power_detectors': ('rms', 'peak'),
        'cyclic_statistics': ('min', 'mean', 'max'),
    },
    'persistence_spectrum': {
        'window': 'hamming',
        'fractional_overlap': 0.5,
        'frequency_resolution': 15e3,
        'persistence_statistics': [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 'max'],
    },
    'channel_power_ccdf': {
        'power_low': -40,
        'power_high': 15,
        'power_resolution': 221,  # 0.25 dB resolution
    },
    'iq_waveform': {'start_time_sec': 0, 'stop_time_sec': 100e-3},
}

# filter_spec = {
#     'nfft': 1024,
#     'window': 'hamming',  # 'hamming', 'blackman', or 'blackmanharris'
# }

# acquisition_spec = {
#     'duration': 0.2,
#     'sample_rate': source.sample_rate
# }

### Single acquisition

In [2]:
# from __future__ import annotations
# import msgspec
# from typing import Literal, Optional
# from channel_analysis import dataarrays
# from pathlib import Path

# class Capture(msgspec.Struct):
#     # RF and leveling
#     center_frequency: float = 3710e6
#     channel: int = 0
#     gain: float = -10
#     calibrated: bool = True

#     # acquisition
#     duration: float = 0.1
#     sample_rate: float = 15.36e6

#     # filtering and resampling
#     analysis_bandwidth: float = 10e6
#     if_frequency: Optional[float] = None # Hz (or none, for no IF frontend)
#     lo_shift: Optional['left'|'right'] = 'left' # shift the LO outside the acquisition band
#     window: 'hamming'|'blackman'|'blackmanharris' = 'hamming' # the COLA spectral window to use


# class System(msgspec.Struct):
#     location: Optional[tuple[str,str,str]] = None
#     timebase: Literal['builtin']|Literal['gps'] = 'builtin'
#     periodic_trigger: bool|float = False
#     calibration: Optional[str] = None
#     defaults: Capture = msgspec.field(default_factory=Capture)

# class Run(msgspec.Struct, omit_defaults=True):
#     acquisition: System = msgspec.field(default_factory=System)
#     sweep: list[Capture] = msgspec.field(default_factory=lambda: [Capture()])
#     channel_analysis: waveform._ConfigStruct = \
#         msgspec.field(default_factory=lambda: waveform._registry.tostruct()())

# runner = Run()
# runner.acquisition.calibration

In [3]:
from channel_analysis import analyze_by_spec

iq = channel_analysis.simulated_awgn(capture)

analyze_by_spec(iq, capture, spec=analysis_spec)

<xarray.Dataset> Size: 25MB
Dimensions:                    (power_detector: 2, time_elapsed: 20,
                                channel_power_bin: 1, cyclic_statistic: 3,
                                cyclic_lag: 600, iq_index: 1536000,
                                persistence_statistic: 9,
                                baseband_frequency: 667)
Coordinates:
  * power_detector             (power_detector) object 16B 'rms' 'peak'
  * time_elapsed               (time_elapsed) float32 80B 0.0 0.01 ... 0.18 0.19
  * channel_power_bin          (channel_power_bin) float32 4B -40.0
  * cyclic_statistic           (cyclic_statistic) object 24B 'min' 'mean' 'max'
  * cyclic_lag                 (cyclic_lag) float32 2kB 0.0 ... 0.009983
  * iq_index                   (iq_index) float64 12MB 0.0 1.0 ... 1.536e+06
  * persistence_statistic      (persistence_statistic) object 72B '0.1' ... '...
  * baseband_frequency         (baseband_frequency) float64 5kB -4.995e+06 .....
Data variables:
    channel_power_time_series  (power_detector, time_elapsed) float32 160B 0....
    channel_power_ccdf         (channel_power_bin) float32 4B 0.9999
    cyclic_channel_power       (power_detector, cyclic_statistic, cyclic_lag) float32 14kB ...
    iq_waveform                (iq_index) complex64 12MB (0.34878474-0.109135...
    persistence_spectrum       (persistence_statistic, baseband_frequency) float32 24kB ...
Attributes:
    duration:            0.2
    sample_rate:         15360000.0
    analysis_bandwidth:  10000000.0
    analysis_filter:     {'nfft': 8192, 'window': 'hamming', 'nfft_out': None}

## RF parameter sweep
### Single parameter

In [5]:
data = []

for fc in [3705e6, 3715e6, 3725e6]:
    iq = channel_analysis.simulated_awgn(capture)

    ret = analyze_by_spec(iq, capture, spec=analysis_spec)

    data.append(ret.assign_coords({'center_frequency': [fc]}))

data = xr.combine_by_coords(data)
data

<xarray.Dataset> Size: 49MB
Dimensions:                    (power_detector: 2, time_elapsed: 20,
                                center_frequency: 3, channel_power_bin: 1,
                                cyclic_statistic: 3, cyclic_lag: 600,
                                iq_index: 1536000, persistence_statistic: 9,
                                baseband_frequency: 667)
Coordinates:
  * power_detector             (power_detector) object 16B 'rms' 'peak'
  * time_elapsed               (time_elapsed) float32 80B 0.0 0.01 ... 0.18 0.19
  * channel_power_bin          (channel_power_bin) float32 4B -40.0
  * cyclic_statistic           (cyclic_statistic) object 24B 'min' 'mean' 'max'
  * cyclic_lag                 (cyclic_lag) float32 2kB 0.0 ... 0.009983
  * iq_index                   (iq_index) float64 12MB 0.0 1.0 ... 1.536e+06
  * persistence_statistic      (persistence_statistic) object 72B '0.1' ... '...
  * baseband_frequency         (baseband_frequency) float64 5kB -4.995e+06 .....
  * center_frequency           (center_frequency) float64 24B 3.705e+09 ... 3...
Data variables:
    channel_power_time_series  (center_frequency, power_detector, time_elapsed) float32 480B ...
    channel_power_ccdf         (center_frequency, channel_power_bin) float32 12B ...
    cyclic_channel_power       (center_frequency, power_detector, cyclic_statistic, cyclic_lag) float32 43kB ...
    iq_waveform                (center_frequency, iq_index) complex64 37MB (0...
    persistence_spectrum       (center_frequency, persistence_statistic, baseband_frequency) float32 72kB ...
Attributes:
    duration:            0.2
    sample_rate:         15360000.0
    analysis_bandwidth:  10000000.0
    analysis_filter:     {'nfft': 8192, 'window': 'hamming', 'nfft_out': None}

In [ ]:
# data = []

# for atten in [0.0, 10.0]:
#     for fc in [3705e6, 3715e6, 3725e6]:
#         iq = simulated_awgn(capture)

#         ret = (
#             dataset.from_spec(
#                     iq,
#                     source,
#                     analysis_spec=analysis_spec,
#             ).assign_coords(
#                 {'center_frequency': [fc], 'attenuation': [atten]}
#             )
#         )

#         data.append(ret)

# ds = xr.combine_by_coords(data)
# ds.center_frequency.attrs = {'label': 'RF center frequency', 'units': 'Hz'}
# ds.attenuation.attrs = {'label': 'External attenuation setting', 'units': 'dB'}

In [ ]:
# (
#     ds
#     .chunk({'iq_sample': round(source.sample_rate*10e-3)})
#     .to_zarr('dataset.zarr', mode='w')
# )

# ds

In [ ]:
# from channel_analysis import figures
# import matplotlib.pyplot as plt
# import iqwaveform # needed for the ieee double column style
# plt.style.use('iqwaveform.ieee_double_column')

# fig, ax = plt.subplots()
# apd = ds.amplitude_probability_distribution.sel(center_frequency=3705e6)
# apd.plot.line(x= 'channel_power_bin', ax=ax)
# ax.set_yscale('gamma-qq', k=1, db_ordinal=True)
# figures.label_axis(ax.xaxis, apd,  'channel_power_bin', tick_units=False)
# figures.label_axis(ax.yaxis, apd)